In [1]:
import torch
import torch.nn as nn
import torchvision

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

from torch.utils.data import Dataset, DataLoader
# from torch.utils.data.sampler import Sampler
import torch.optim as optim
import sys
sys.path.append('../')

from dataset import LbpDataset, train_transforms, val_transforms, test_transforms, collate_fn, get_data
from visualize import visualize
from model import fasterrcnn_resnet201_fpn, FastRCNNPredictor
from engine import evaluate
import utils
from train_lbp import get_train_test_list

In [2]:
from torchvision.models.detection.retinanet import retinanet_resnet50_fpn
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
import easydict 
args = easydict.EasyDict({ "batch_size": 8, 
                          "epochs": 50, 
                          "data": 0, 
                          'lr':0.1,
                         'momentum':0.9,
                         'weight_decay':1e-4,
                         'start_epoch':0,
                         'gpu':5,
                          'workers':4,
                         'output_dir' :'../trained_models/retinanet/'})

In [4]:
df = pd.read_csv('../../data/df.csv')
df.head()
# Data loading code
data_dir = '../../data/df.csv'
train_list, test_list = get_train_test_list(data_dir)
train_dataset = LbpDataset(train_list, transform=train_transforms)
test_dataset = LbpDataset(test_list, transform=val_transforms)  

total 4019 train 3014 test 1005
3014
1005


In [5]:
train_sampler = torch.utils.data.RandomSampler(train_dataset)
test_sampler = torch.utils.data.SequentialSampler(test_dataset)

train_loader = DataLoader(
    train_dataset, batch_size=args.batch_size,
    sampler=train_sampler, num_workers=args.workers,
    collate_fn=utils.collate_fn)

test_loader = DataLoader(
    test_dataset, batch_size=args.batch_size,
    sampler=test_sampler, num_workers=args.workers,
    collate_fn=utils.collate_fn)

In [6]:
num_classes = 2
pretrained = retinanet_resnet50_fpn(pretrained=True)
model = retinanet_resnet50_fpn(pretrained=False, min_size=1024, max_size=1024, num_classes=num_classes)
model.backbone.load_state_dict(pretrained.backbone.state_dict())
model.head.regression_head.load_state_dict(pretrained.head.regression_head.state_dict())
device = torch.device('cuda')
model.to(device)
print('model is loaded to gpu')

model is loaded to gpu


In [7]:
# model.eval()

In [8]:
params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.Adam(params, lr=0.0001)
optimizer = torch.optim.SGD(
       params, lr=0.001, momentum=0.9, weight_decay=1e-4)
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[15, 30, 45, 60, 75], 
                                                    gamma=0.5)

In [9]:
# evaluate(model, test_loader, device=device)  
checkpoint = torch.load(args.output_dir + 'checkpoint.pth')
state_dict = checkpoint['model']
model.load_state_dict(state_dict)

<All keys matched successfully>

In [10]:
from train_lbp import train_one_epoch

start_time = time.time()
for epoch in range(90):
    train_one_epoch(model, optimizer, train_loader, device, epoch, 400)
    lr_scheduler.step()
    
    if epoch > 20 and epoch % 5 == 0 :
        if args.output_dir:
            checkpoint = {
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'lr_scheduler': lr_scheduler.state_dict(),
                'args': args,
                'epoch': epoch
            }
            utils.save_on_master(
                checkpoint,
                os.path.join(args.output_dir, 'model_{}.pth'.format(epoch)))
            utils.save_on_master(
                checkpoint,
                os.path.join(args.output_dir, 'checkpoint.pth'))

        # evaluate after every epoch
        evaluate(model, test_loader, device=device)    
print('total time is {}'.format(time.time() - start_time))    

Epoch: [0]  [  0/377]  eta: 0:43:04  lr: 0.001000  loss: 1.1707 (1.1707)  classification: 0.7110 (0.7110)  bbox_regression: 0.4597 (0.4597)  time: 6.8544  data: 2.9535  max mem: 7675
Epoch: [0]  [376/377]  eta: 0:00:00  lr: 0.001000  loss: 1.3584 (1.3740)  classification: 0.8135 (0.8209)  bbox_regression: 0.5435 (0.5531)  time: 0.9125  data: 0.0382  max mem: 7921
Epoch: [0] Total time: 0:05:51 (0.9323 s / it)
Epoch: [1]  [  0/377]  eta: 0:24:09  lr: 0.001000  loss: 1.4568 (1.4568)  classification: 0.8491 (0.8491)  bbox_regression: 0.6078 (0.6078)  time: 3.8457  data: 2.9538  max mem: 7921
Epoch: [1]  [376/377]  eta: 0:00:00  lr: 0.001000  loss: 1.3505 (1.3649)  classification: 0.8272 (0.8208)  bbox_regression: 0.5416 (0.5441)  time: 0.8964  data: 0.0330  max mem: 7921
Epoch: [1] Total time: 0:05:48 (0.9252 s / it)
Epoch: [2]  [  0/377]  eta: 0:25:17  lr: 0.001000  loss: 1.4314 (1.4314)  classification: 0.8931 (0.8931)  bbox_regression: 0.5383 (0.5383)  time: 4.0245  data: 3.1397  max m

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


Test:  [  0/126]  eta: 0:06:16  model_time: 0.6921 (0.6921)  evaluator_time: 0.0770 (0.0770)  time: 2.9854  data: 2.1478  max mem: 7921
Test:  [100/126]  eta: 0:00:16  model_time: 0.4642 (0.4645)  evaluator_time: 0.0337 (0.0523)  time: 0.5874  data: 0.0294  max mem: 7921


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


Test:  [125/126]  eta: 0:00:00  model_time: 0.4601 (0.4624)  evaluator_time: 0.0325 (0.0497)  time: 0.5705  data: 0.0286  max mem: 7921
Test: Total time: 0:01:17 (0.6128 s / it)
Averaged stats: model_time: 0.4601 (0.4624)  evaluator_time: 0.0325 (0.0497)
Accumulating evaluation results...
DONE (t=1.55s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.018
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.047
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.005
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.081
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.042
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.165
 Average Recall     (AR) @[ IoU=0.50:

...

Epoch: [87]  [  0/377]  eta: 0:16:46  lr: 0.000031  loss: 0.9506 (0.9506)  classification: 0.5507 (0.5507)  bbox_regression: 0.3999 (0.3999)  time: 2.6692  data: 1.9905  max mem: 7921
Epoch: [87]  [376/377]  eta: 0:00:00  lr: 0.000031  loss: 0.9600 (0.9427)  classification: 0.5554 (0.5627)  bbox_regression: 0.3909 (0.3800)  time: 0.6966  data: 0.0379  max mem: 7921
Epoch: [87] Total time: 0:04:24 (0.7026 s / it)


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


Epoch: [88]  [  0/377]  eta: 0:16:50  lr: 0.000031  loss: 0.8801 (0.8801)  classification: 0.5427 (0.5427)  bbox_regression: 0.3374 (0.3374)  time: 2.6797  data: 2.0192  max mem: 7921
Epoch: [88]  [376/377]  eta: 0:00:00  lr: 0.000031  loss: 0.8716 (0.9409)  classification: 0.5394 (0.5607)  bbox_regression: 0.3493 (0.3802)  time: 0.6905  data: 0.0281  max mem: 7921
Epoch: [88] Total time: 0:04:25 (0.7049 s / it)


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


Epoch: [89]  [  0/377]  eta: 0:16:15  lr: 0.000031  loss: 0.9420 (0.9420)  classification: 0.5894 (0.5894)  bbox_regression: 0.3526 (0.3526)  time: 2.5880  data: 1.9205  max mem: 7921
Epoch: [89]  [376/377]  eta: 0:00:00  lr: 0.000031  loss: 0.9370 (0.9394)  classification: 0.5424 (0.5591)  bbox_regression: 0.3808 (0.3803)  time: 0.6870  data: 0.0281  max mem: 7921
Epoch: [89] Total time: 0:04:24 (0.7005 s / it)
total time is 31312.228939294815


In [11]:
evaluate(model, test_loader, device=device) 

creating index...
index created!


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)


Test:  [  0/126]  eta: 0:05:13  model_time: 0.4888 (0.4888)  evaluator_time: 0.0631 (0.0631)  time: 2.4860  data: 1.8805  max mem: 7921
Test:  [100/126]  eta: 0:00:12  model_time: 0.3296 (0.3345)  evaluator_time: 0.0310 (0.0355)  time: 0.4481  data: 0.0297  max mem: 7921
Test:  [125/126]  eta: 0:00:00  model_time: 0.3290 (0.3326)  evaluator_time: 0.0309 (0.0345)  time: 0.4322  data: 0.0287  max mem: 7921
Test: Total time: 0:00:58 (0.4642 s / it)
Averaged stats: model_time: 0.3290 (0.3326)  evaluator_time: 0.0309 (0.0345)
Accumulating evaluation results...
DONE (t=0.91s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.037
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.093
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.021
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Aver